<a href="https://colab.research.google.com/github/benedettacandelori/Monkeys_go_stop_trials/blob/main/Monkeys_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy.io
import os
import math
import torch

In [ ]:
os.chdir('/content/drive/MyDrive')
os.getcwd() 

'/content/drive/MyDrive'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
%pip install pytorch_lightning einops seaborn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.2 MB/s eta 0:00:00


In [ ]:
import math
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pylight
from einops.layers.torch import Rearrange
from einops import reduce, rearrange, repeat
from torchmetrics.classification import Accuracy, F1Score, MulticlassAccuracy

# Dataset

### Dataset creation

In [ ]:
class Monkeys_dataset(Dataset):
    """Monkeys dataset"""

    def __init__(self, mat_file_labels, mat_file_x, root_dir, start_idx = 800, end_idx = 1200):
        """
        Args:
            mat_file (string): Path to the .mat files.
            root_dir (string): Directory with all the data.
        """
        self.classes = scipy.io.loadmat(mat_file_labels)['nostop']
        self.data = scipy.io.loadmat(mat_file_x)['matrix']
        self.root_dir = root_dir
        self.start_idx = start_idx
        self.end_idx = end_idx

    def __len__(self):
        return len(self.classes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        x = self.data[:,idx,self.start_idx:self.end_idx]
        x = torch.tensor(x).float().squeeze(0)#.unsqueeze(1) 
        label = int(self.classes[idx,1])
      
        #sample = {'matrix': x, 'class': label}
        sample = (x, label)

        return sample

In [ ]:
dataset = Monkeys_dataset(mat_file_labels = 'Stop_Task_Collab_2023/BHV_Go.mat', mat_file_x = 'Stop_Task_Collab_2023/activity_movement_onset_Go.mat',
                                    root_dir='Stop_Task_Collab_2023/')


In [ ]:
print(dataset[0][0].shape)
print(dataset[0][1])

torch.Size([105, 400])
2


### Data loader

In [ ]:
# We split the dataset in training and test portions
monkey_train, monkey_test = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [ ]:
# Loaders create mini-batches of images from the original dataset
loader_train = torch.utils.data.DataLoader(monkey_train, batch_size=32, shuffle=True)
loader_test = torch.utils.data.DataLoader(monkey_test, batch_size=32, shuffle=False)

In [ ]:
for xb, yb in loader_train:
  break

In [ ]:
print(xb.shape)
print(yb.shape)

torch.Size([32, 105, 400])
torch.Size([32])


# Transformer architecture


In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


In [ ]:
class SeqEmbedding(nn.Module):
  def __init__(self, inp_shape, d_output):
    super().__init__()
    self.stem = nn.Sequential(nn.Conv1d(inp_shape, d_output//2, 5), nn.ReLU(), nn.Conv1d(d_output//2, d_output, 5))
    self.pos_enc = PositionalEncoding(d_output)
    
  def forward(self, x):
    x = self.stem(x) 
    x = rearrange(x,'b h seq -> seq b h')
    x = self.pos_enc(x)
    return x#.unsqueeze(0) # aggiunge la prima dimensione

In [ ]:
print(xb.shape)
xb_reshape = rearrange(xb, 'b h w -> h b w')
print(xb_reshape.shape)

torch.Size([32, 105, 400])
torch.Size([105, 32, 400])


In [ ]:
print(SeqEmbedding(105,16)(xb).shape)
rearrange(SeqEmbedding(105,16)(xb), 'b h w -> h w b').shape

torch.Size([392, 32, 16])


torch.Size([32, 16, 392])

In [ ]:
class SelfMHA(nn.Module):
  def __init__(self, d_input, d_value, d_query, d_output, n_heads=8, dropout=0.0):
      super().__init__()
      self.n_heads = n_heads
      self.query = nn.Linear(d_input, d_query * n_heads)
      self.keys = nn.Linear(d_input, d_query * n_heads)
      self.values = nn.Linear(d_input, d_value * n_heads)
      self.output = nn.Linear(d_value * n_heads, d_output)
      self.drop = nn.Dropout(dropout)

  def forward(self, x):
      Q = rearrange(self.query(x), 'b seq (q heads) -> b heads seq q', heads = self.n_heads)

      K = rearrange(self.keys(x), 'b seq (q heads) -> b heads seq q', heads = self.n_heads)
      V = rearrange(self.values(x), 'b seq (v heads) -> b heads seq v', heads = self.n_heads)

      A = torch.einsum('b h s q, b h t q -> b h s t', [Q, K]) / math.sqrt(Q.shape[-1])
      A = torch.softmax(A, dim=3)

      O = torch.einsum('b h s t, b h t v -> b h s v', [A, V])
      O = rearrange(O, 'b heads seq v -> b seq (heads v)')
      return self.drop(self.output(O)), A

In [ ]:
SelfMHA(d_input = 400, d_value = 16, d_query = 16,d_output= 400, n_heads=6, dropout=0.)(xb)[0].shape

torch.Size([32, 105, 400])

In [ ]:
xb_rearr = rearrange(SeqEmbedding(105,16)(xb), 'b h w -> h b w')
print(xb_rearr.shape)
SelfMHA(d_input = 16, d_value = 392, d_query = 392,d_output= 16, n_heads=6, dropout=0.)(xb_rearr)[0].shape

torch.Size([32, 392, 16])


torch.Size([32, 392, 16])

In [ ]:
print(xb.shape)
print(xb_rearr.shape)

torch.Size([32, 105, 400])
torch.Size([32, 16, 392])


In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self, d_input, d_hidden):
    super().__init__()
    self.sa = SelfMHA(d_input, d_hidden, d_hidden, d_input, n_heads=6, dropout=0.)
    self.ln1 = nn.LayerNorm(d_input)
    self.ln2 = nn.LayerNorm(d_input)
    self.dense = nn.Sequential(
        nn.Linear(d_input, d_hidden),
        nn.GELU(),
        nn.Linear(d_hidden, d_input)
    )

  def forward(self, x):
    x_t, a = self.sa(self.ln1(x))
    x = x_t + x
    x = self.dense(self.ln2(x)) + x
    return x, a

In [ ]:
TransformerBlock(400,400)(xb)[0].shape

torch.Size([32, 105, 400])

In [ ]:
TransformerBlock(392,392)(xb_rearr)[0].shape

torch.Size([32, 16, 392])

In [ ]:
from torch.functional import align_tensors
class Transformer(pylight.LightningModule):

  def __init__(self, embedding: nn.Module, n_classes=10, hidden_dimension=128):
    super().__init__()

    self.input_embedding = embedding
    self.time = TransformerBlock(hidden_dimension, hidden_dimension)
    self.space = TransformerBlock(hidden_dimension, hidden_dimension)
    self.classifier = nn.Linear(2*hidden_dimension, n_classes)
    self.acc = MulticlassAccuracy(n_classes)

  def forward(self, x, mask=None):

    x = self.input_embedding(x)
    x = rearrange(x, 'b h w -> h b w')
    #x = rearrange(x, 'b s n h -> (b s) n h')
    x, a_s = self.space(x)
    #x = rearrange(x, '(b s) n h -> (b n) s h', s=392)
    x, a_t = self.time(x)
    #x = rearrange(x, '(b n) s h -> b s n h', n=105)
    x_mean = reduce(x, 'b  s h -> b h', reduction='mean')
    x_max = reduce(x, 'b  s h -> b h', reduction='max')
    x = torch.cat((x_mean, x_max), dim=-1)

    return self.classifier(x), a_s, a_t

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters())

  def training_step(self, batch, _):
    xb, yb = batch
    y_pred, a_s, a_t = self(xb)
    loss = F.cross_entropy(y_pred, yb)
    self.acc(y_pred, yb)
    self.log("train/loss", loss, on_step=True, on_epoch=False, prog_bar=True)
    self.log("train/acc", self.acc, on_step=True, on_epoch=False, prog_bar=True)
    return loss#, a_s, a_t

  def validation_step(self, batch, _):
    xb, yb = batch
    y_pred, a_s, a_t = self(xb)
    self.acc(y_pred, yb)
    self.log("val/acc", self.acc, on_step=True, on_epoch=True, prog_bar=True)

  def test_step(self, batch, _):
    xb, yb = batch
    y_pred, a_s, a_t = self(xb)
    self.acc(y_pred, yb)
    self.log("test/acc", self.acc, on_step=True, on_epoch=True, prog_bar=True)

# Training the model

In [ ]:
#enc = SeqEmbedding(1, 16)
enc = SeqEmbedding(105,16)
t = Transformer(enc, n_classes=3, hidden_dimension=16)

In [ ]:
trainer = pylight.Trainer(max_epochs=10, accelerator='gpu', devices=1)
model = trainer.fit(t, loader_train, loader_test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type               | Params
-------------------------------------------------------
0 | input_embedding | SeqEmbedding       | 4.9 K 
1 | time            | TransformerBlock   | 7.1 K 
2 | space           | TransformerBlock   | 7.1 K 
3 | classifier      | Linear             | 99    
4 | acc             | MulticlassAccuracy | 0     
-------------------------------------------------------
19.1 K    Trainable params
0         Non-trainable params
19.1 K    Total params
0.076     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:

# Save trained model as a checkpoint-
trainer.save_checkpoint("results_Trained.ckpt")

In [ ]:
trained_model = Transformer.load_from_checkpoint(
    "results_Trained.ckpt",embedding = enc, n_classes=3, hidden_dimension=16)

In [ ]:
trained_model

In [ ]:
model.load_from_checkpoint('/content/drive/MyDrive/Stop_Task_Collab_2023/logs/lightning_logs/version_1/checkpoints/epoch=9-step=200-v1.ckpt')


In [ ]:
trainer.test(model,dataloaders=loader_test)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/.shortcut-targets-by-id/1h56UfP3ks19H6m2iMcS2G9c-yMnwk0bM/Stop_Task_Collab_2023/lightning_logs/version_24/checkpoints/epoch=9-step=200.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/drive/.shortcut-targets-by-id/1h56UfP3ks19H6m2iMcS2G9c-yMnwk0bM/Stop_Task_Collab_2023/lightning_logs/version_24/checkpoints/epoch=9-step=200.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val/acc_epoch         0.6627451181411743
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val/acc_epoch': 0.6627451181411743}]

In [ ]:


logged_metrics = trainer.logged_metrics



4